<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2024-1/Insumos/OpcionesTasa_capfloor_1flujo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

#Insumos

In [ ]:
%%R
btasadesc="tasa_TIIE_SW_OP.txt"
btasaspot="tasa_DIRS_SW_OP.txt"
bvolspot="tvoltiie_opc.txt"
direc="RiesgosFinancieros/2021-1/Insumos/"
plazos=cbind( 39,	115,	153,	245,	360) #T-t
pr=28 #plazo de referencia
dct=360 #d_base
cp=cbind(1,0,0,0,0) #si es call (cap) o put (floor)
K=cbind( 0.07, 0.085, 0.078, 0.085, 0.08)
contratos=cbind(2200000, -2900000, 2900000, -4600000, 5000)
nominal=1
cs=1 #1 si es continua la tasa 0 si es simple
itpl=0 #interpolación lineal o 1 alambrada

#direc=directorio siempre entre " " y cambiar la "/" por "/"
#btasadesc=nombre de base para descontar a importar siempre entre " "
#btasaspot=nombre de base para spot a importar siempre entre " "
#bvolspot=nombre de base para volatilidad de spot a importar siempre entre " "
#plazos= vector de plazos
#contratos=vector de contratos
#K=vector de strikes
#alpha= nivel de confianza
#nominal=vector de nominales


#Carga de información

In [ ]:
%%R
#setwd(direc)
#carga de datos
#carga de rho
data1<-read.table(btasadesc)
n<-nrow(data1)
m1_orig=ncol(data1)
x1_orig=data.frame(data1[2:n,2:m1_orig])/100
nodos1=data.frame(data1[1,2:m1_orig])
print(nodos1)

print(x1_orig[1,])
#carga de tasas spot
data2<-read.table(btasaspot)
n<-nrow(data2)
m2_orig=ncol(data2)
x2_orig=data.frame(data2[2:n,2:m2_orig])/100
nodos2=data.frame(data2[1,2:m2_orig])
print(nodos2)
print(x2_orig[1,])
#carga de volatilidades de spot
data3<-read.table(bvolspot)
n<-nrow(data3)
m3_orig=ncol(data3)
x3_orig=data.frame(data3[2:n,2:m3_orig])
nodos3=data.frame(data3[1,2:m3_orig])
print(nodos3)
x3_orig[1,]

#Función interpolación alambrada

In [ ]:
%%R
#Función de interpolación alambrada
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


# Interpolación de tasas y volatilidades

In [ ]:
%%R
m=ncol(plazos)
x1=matrix(0,n-1,m)
x2tc=matrix(0,n-1,m)
x2tl=matrix(0,n-1,m)
x2=matrix(0,n-1,m)
x3=matrix(0,n-1,m)
for (i in 1:(n-1))
{
x1[i,]=if(itpl==0){approx(nodos1,x1_orig[i,],plazos)$y}else{talamb(nodos1,x1_orig[i,],plazos)}
x2tc[i,]=if(itpl==0){approx(nodos2,x2_orig[i,],plazos)$y}else{talamb(nodos2,x2_orig[i,],plazos)}
x2tl[i,]=if(itpl==0){approx(nodos2,x2_orig[i,],(plazos+pr))$y}else{talamb(nodos2,x2_orig[i,],(plazos+pr))}
x3[i,]=if(itpl==0){approx(nodos3,x3_orig[i,],plazos)$y}else{talamb(nodos3,x3_orig[i,],plazos)}
x2[i,]=((1+x2tl[i,]*(plazos+pr)/360)/(1+x2tc[i,]*(plazos)/360)-1)*360/pr
}

#Posición Inicial

In [ ]:
%%R
x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

print(x01)
print(x02)
print(x03)

#Función de opciones de tasas de interés


In [ ]:
%%R
opctint = function(d,S,K,vol,t,cp,cs,pr,dct)	#función de una opción europea
{
  d1=if(cs==1){(log(S/K)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs==1){(log(S/K)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs==1){log(1+d*t/360)*365/t}else{d}
  (if(cs==1){(S*pnorm(d1*(-1)^cp)-K*pnorm(d2*(-1)^cp))*(exp(-vp*t/365))*(-1)^cp}else{(S*pnorm(d1*(-1)^cp)-K*pnorm(d2*(-1)^cp))*(1/(1+vp*t/360))*(-1)^cp})*(pr/dct)/(1+S*pr/dct)
}


V0=opctint(x01,x02,K,x03,plazos,cp,cs,pr,dct)*contratos*nominal #Valor del portafolio
V0

## Valuación Opciones

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd

#OPCIONES
btasadesc_oir = "RiesgosFinancieros/2020-2/Insumos/tasa_TIIE_SW_OP.txt"
btasaspot_oir = "RiesgosFinancieros/2020-2/Insumos/tasa_DIRS_SW_OP.txt"
bvolspot_oir = "RiesgosFinancieros/2020-2/Insumos/tvoltiie_opc.txt"
plazos_oir = np.array([700, 1700, 400, 350, 200])  # T-t
pr_oir = 28  # plazo de referencia
dct_oir = 360  # d_base
cp_oir = np.array([1, 0, 0, 0, 0])  # si es call (cap) o put (floor)
K_oir = np.array([0.07, 0.085, 0.078, 0.085, 0.08])
contratos_oir = np.array([2200, -2900, 2900, -460, 5000])
nominal_oir = 1
cs_oir = 1  # 1 si es continua la tasa 0 si es simple

#carga de opciones de tasa de interés
data1 = pd.read_table(btasadesc_oir,header=None)
n =data1.shape[0]
m1_orig_oir = len(data1.columns)
x1_orig_oir = pd.DataFrame(data1.iloc[1:n, 1:m1_orig_oir], dtype=float)
# x1_orig_oir['Date'] = pd.to_datetime(data1.iloc[1:n, 0], format = '%Y%m%d')
nodos1_oir =  data1.iloc[0, 1:m1_orig_oir]

# carga de tasas spot
data2 = pd.read_table(btasaspot_oir,header=None)
n = len(data2)
m2_orig_oir = len(data2.columns)
x2_orig_oir = pd.DataFrame(data2.iloc[1:n, 1:m2_orig_oir], dtype=float)
# x2_orig_oir['Date'] = pd.to_datetime(data2.iloc[1:n, 0], format="%Y%m%d")
nodos2_oir = data2.iloc[0, 1:m2_orig_oir]

# carga de volatilidades de spot
data3 = pd.read_table(bvolspot_oir,header=None)
n = len(data3)
m3_orig_oir = len(data3.columns)
x3_orig_oir = pd.DataFrame(data3.iloc[1:n, 1:m3_orig_oir], dtype=float)
# x3_orig_oir['Date'] = pd.to_datetime(data3.iloc[1:n, 0], format="%Y%m%d")
nodos3_oir = data3.iloc[0, 1:m3_orig_oir]

#x1[i, :] = np.interp(plazos_oir, nodos1_oir.index, x1_orig_oir.iloc[i, :-1].astype("float64")/100)
# print( x1_orig_oir.iloc[i, :-1].astype("float64"))
print(nodos1_oir.index)



# Opciones de tasa de interés, cálculo con inicio el día de la valuación

# Posición inicial

# Interpolación de tasas y volatilidades
m = len(plazos_oir)
x1 = np.zeros((n, m))
x2tc = np.zeros((n, m))
x2tl = np.zeros((n, m))
x2 = np.zeros((n, m))
x3 = np.zeros((n, m))
for i in range(n-1):
    x1[i, :] = np.interp(plazos_oir, nodos1_oir, x1_orig_oir.iloc[i, :])
    x2tc[i, :] = np.interp(plazos_oir, nodos2_oir, x2_orig_oir.iloc[i, :])
    x2tl[i, :] = np.interp(plazos_oir + pr_oir, nodos2_oir, x2_orig_oir.iloc[i, :])
    x3[i, :] = np.interp(plazos_oir, nodos3_oir, x3_orig_oir.iloc[i, :])
    x2[i, :] = (((1 + x2tl[i, :] * ((plazos_oir + pr_oir) / 360)) / (1 + x2tc[i, :] * (plazos_oir / 360))) - 1) * 360 / pr_oir

x01 = x1[0, :]  # Tasas de descuento
x02 = x2[0, :]  # Tasas spot
x03 = x3[0, :]  # Volatilidades


X_oir = np.column_stack((x1, x2, x3))
def opctint(d, S, K_oir, vol, t, cp_oir, cs_oir, pr_oir, dct_oir):
    d1 = ((np.log(S / K_oir) + (vol ** 2) * t / (365 * 2)) * (1 / (vol * np.sqrt(t / 365)))) if cs_oir == 1 else ((np.log(S / K_oir) + (vol ** 2) * t / (360 * 2)) * (1 / (vol * np.sqrt(t / 360))))
    d2 = ((np.log(S / K_oir) - (vol ** 2) * t / (365 * 2)) * (1 / (vol * np.sqrt(t / 365)))) if cs_oir == 1 else ((np.log(S / K_oir) - (vol ** 2) * t / (360 * 2)) * (1 / (vol * np.sqrt(t / 360))))
    vp = (np.log(1 + d * t / 360) * 365 / t) if cs_oir == 1 else d
    f=((S * stats.norm.cdf(d1 * (-1) ** cp_oir) - K_oir * stats.norm.cdf(d2 * (-1) ** cp_oir)) * (np.exp(-vp * t / 365)) * (-1) ** cp_oir)* (pr_oir / dct_oir) / (1 + S * pr_oir / dct_oir) if cs_oir == 1 else ((S * stats.norm.cdf(d1 * (-1) ** cp_oir) - K_oir * stats.norm.cdf(d2 * (-1) ** cp_oir)) * (1 / (1 + vp * t / 360)) * (-1) ** cp_oir) * (pr_oir / dct_oir) / (1 + S * pr_oir / dct_oir)
    return f

V0_oir = opctint(x01, x02, K_oir, x03, plazos_oir, cp_oir, cs_oir, pr_oir, dct_oir) * contratos_oir * nominal_oir  # Valor del portafolio
V0_oir
